In [0]:
%run "../includes/configuration"

In [0]:
v_source_file = dbutils.widgets.get("p_source_file")

In [0]:
from pyspark.sql.types import StructType,StructField,StringType,IntegerType,DoubleType,LongType,TimestampType

In [0]:
pit_stops_schama = StructType(fields=[StructField("raceId", IntegerType(), True),
                                    StructField("driverId", IntegerType(), False),
                                    StructField("stop", IntegerType(), False),
                                    StructField("lap", IntegerType(), False),
                                    StructField("time", TimestampType(), False),
                                    StructField("duration", StringType(), False),
                                    StructField("milliseconds", IntegerType(), False)])

In [0]:
results_df = spark.read \
    .option("multiline", "true") \
    .schema(pit_stops_schama) \
    .json(f"{raw_folder_path}/pit_stops.json")

# results_df = spark.read \
#     .option("header", "true") \
#     .option("inferSchema", "true") \
#     .json("/mnt/dldatabrickscoursedev001/raw/results.json")



In [0]:
from pyspark.sql.functions import col, current_timestamp , lit

In [0]:
results_columnAlteration_df = results_df.withColumnRenamed("raceId", "race_id") \
                                        .withColumnRenamed("driverId", "driver_id") \
                                        .withColumn("data_source",lit(v_source_file))

In [0]:
%run "../functions/common_functions"

In [0]:
results_columnAlteration_df = add_ingest_date(results_columnAlteration_df)

In [0]:
results_columnAlteration_df.write \
    .mode("overwrite")  \
    .parquet(f"{processed_folder_path}/pit_stops")

In [0]:
spark.read.parquet(f'{processed_folder_path}/pit_stops').display()

In [0]:
dbutils.notebook.exit("success")